In [107]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics.pairwise import cosine_similarity
import xgboost as xgb

In [94]:
FAU = ["AU01", "AU02", "AU04", "AU06", "AU12", "AU15", "AU17", "AU20", "AU23"]
len(FAU)

9

In [3]:
df = pd.read_csv("./OpenFace/output/russell_test.csv")
df.head()

,frame,face_id,timestamp,confidence,success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.98,1,0.0,0.0,0.11,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,0.033,0.98,1,0.0,0.0,0.38,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,0.067,0.98,1,0.0,0.0,0.42,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0.100,0.98,1,0.0,0.0,0.23,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0.133,0.98,1,0.0,0.0,0.07,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
df_fau = pd.DataFrame(df.iloc[:, 5:])
df_fau.columns = [c.strip() for c in df_fau.columns]
df_fau

,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,0.00,0.0,0.11,0.00,0.00,0.00,0.00,0.00,0.40,0.00,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00,0.0,0.38,0.00,0.00,0.00,0.00,0.00,0.41,0.16,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00,0.0,0.42,0.00,0.00,0.07,0.00,0.00,0.40,0.16,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.0,0.23,0.00,0.00,0.07,0.00,0.00,0.43,0.16,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00,0.0,0.07,0.00,0.00,0.07,0.00,0.00,0.49,0.07,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,1.92,0.0,0.00,0.27,0.75,0.00,0.87,0.70,1.04,2.02,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
140,1.35,0.0,0.00,0.15,0.47,0.00,0.61,0.53,0.70,1.47,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
141,0.82,0.0,0.00,0.06,0.33,0.00,0.97,0.33,0.88,1.53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142,0.13,0.0,0.00,0.06,0.34,0.00,1.36,0.00,0.95,1.49,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [61]:
cond1 = df_fau[" AU01_c"] == 1
cond2 = df_fau[" AU01_r"] >= 0.5
df_fau["AU01"] = np.where(cond1 & cond2, 1, 0)
df_fau["AU01"]

0      0
1      0
2      0
3      0
4      0
      ..
139    0
140    0
141    0
142    0
143    0
Name: AU01, Length: 144, dtype: int32

In [92]:
# import the cleaned FAU data
df_fau_cleaned = pd.read_csv("fau_cleaned.csv")
df_fau_cleaned.head()

,Subject,Category,AU01_c,AU02_c,AU04_c,AU06_c,AU12_c,AU15_c,AU17_c,AU20_c,AU23_c,Duration(sec)
0,S01,Anxiety,3735,4645,6531,2560,1112,742,1796,2351,11201,279.03
1,S01,Mild,5464,5075,15308,933,1146,1898,3841,3294,14841,362.69
2,S01,Moderate,3769,2820,6281,962,1080,1145,2537,1951,8010,235.54
3,S01,Severe,12185,11592,46926,1935,3387,4025,6645,6988,17164,723.60
4,S02,Anxiety,6019,5963,6415,16026,2386,2757,3088,3384,16087,256.79


### Get the average of each FAU based on the duration of the video

In [126]:
# get the average of each FAU based on the duration of the video 
for fau in FAU:
    df_fau_cleaned[fau + "_avg"] = np.ceil(df_fau_cleaned[fau + "_c"] / df_fau_cleaned["Duration(sec)"])

# extract the different categories 
df_anxiety = np.array(df_fau_cleaned[df_fau_cleaned["Category"] == "Anxiety"].iloc[:, -9:])
df_mild = np.array(df_fau_cleaned[df_fau_cleaned["Category"] == "Mild"].iloc[:, -9:])
df_moderate = np.array(df_fau_cleaned[df_fau_cleaned["Category"] == "Moderate"].iloc[:, -9:])
df_severe = np.array(df_fau_cleaned[df_fau_cleaned["Category"] == "Severe"].iloc[:, -9:])

In [127]:
df_anxiety

array([[14., 17., 24., 10.,  4.,  3.,  7.,  9., 41.],
       [24., 24., 25., 63., 10., 11., 13., 14., 63.],
       [ 6.,  2., 38.,  5.,  6., 11.,  8.,  5., 51.],
       [27., 25., 50., 13.,  4., 15., 15., 18., 20.],
       [18., 22., 32., 33., 13., 17., 18., 12., 13.],
       [ 9.,  5., 26.,  0.,  0., 19., 11.,  9.,  1.],
       [17., 24., 17., 41., 58., 20., 20.,  8., 22.]])

### Getting Cosine Similarity for Russell's video

In [101]:
# import input data
df_russell = pd.read_csv("russell_test.csv")
df_russell.columns = [c.strip() for c in df_russell.columns]
df_russell.head()

,frame,face_id,timestamp,confidence,success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.98,1,0.0,0.0,0.11,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,0.033,0.98,1,0.0,0.0,0.38,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,0.067,0.98,1,0.0,0.0,0.42,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0.100,0.98,1,0.0,0.0,0.23,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0.133,0.98,1,0.0,0.0,0.07,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
# get to duration of the video 
duration = df_russell["timestamp"].iloc[-1]
duration

4.767

In [104]:
for au in FAU:
    cond1 = df_russell[au + "_c"] == 1
    cond2 = df_russell[au + "_r"] >= 0.5
    cond3 = df_russell["confidence"] >= 0.98
    cond4 = df_russell["success"] == 1
    df_russell[au] = np.where(cond1 & cond2 & cond3 & cond4, 1, 0)

df_russell

,frame,face_id,timestamp,confidence,success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,...,AU45_c,AU01,AU02,AU04,AU06,AU12,AU15,AU17,AU20,AU23
0,1,0,0.000,0.98,1,0.00,0.0,0.11,0.00,0.00,...,0.0,0,0,0,0,0,0,0,0,0
1,2,0,0.033,0.98,1,0.00,0.0,0.38,0.00,0.00,...,0.0,0,0,0,0,0,0,0,0,0
2,3,0,0.067,0.98,1,0.00,0.0,0.42,0.00,0.00,...,0.0,0,0,0,0,0,0,0,0,0
3,4,0,0.100,0.98,1,0.00,0.0,0.23,0.00,0.00,...,0.0,0,0,0,0,0,0,0,0,0
4,5,0,0.133,0.98,1,0.00,0.0,0.07,0.00,0.00,...,0.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,140,0,4.633,0.93,1,1.92,0.0,0.00,0.27,0.75,...,1.0,0,0,0,0,0,0,0,0,0
140,141,0,4.667,0.88,1,1.35,0.0,0.00,0.15,0.47,...,1.0,0,0,0,0,0,0,0,0,0
141,142,0,4.700,0.03,0,0.82,0.0,0.00,0.06,0.33,...,0.0,0,0,0,0,0,0,0,0,0
142,143,0,4.733,0.77,1,0.13,0.0,0.00,0.06,0.34,...,0.0,0,0,0,0,0,0,0,0,0


In [106]:
df_russell_filtered = np.ceil(df_russell[FAU].sum() / duration)
df_russell_filtered

AU01     5.0
AU02     3.0
AU04    16.0
AU06     0.0
AU12     2.0
AU15     2.0
AU17     9.0
AU20     1.0
AU23     2.0
dtype: float64

In [113]:
df_russell_array = np.array(df_russell_filtered).reshape(1, -1)
df_russell_array

array([[ 5.,  3., 16.,  0.,  2.,  2.,  9.,  1.,  2.]])

In [131]:
# compare the input vector with all 28 vectors
anxiety_vector = []
mild_vector = []
moderate_vector = []
severe_vector = []
for i in range(len(df_anxiety)):
    anxiety = cosine_similarity(df_anxiety[i].reshape(1, -1), df_russell_array)
    mild = cosine_similarity(df_mild[i].reshape(1, -1), df_russell_array)
    moderate = cosine_similarity(df_moderate[i].reshape(1, -1), df_russell_array)
    severe = cosine_similarity(df_severe[i].reshape(1, -1), df_russell_array)
    anxiety_vector = np.append(anxiety_vector, anxiety)
    mild_vector = np.append(mild_vector, mild)
    moderate_vector = np.append(moderate_vector, moderate)
    severe_vector = np.append(severe_vector, severe)

print("Anxiety:", anxiety_vector)
print("Mild:", mild_vector)
print("Moderate:", moderate_vector)
print("Severe:", severe_vector)

Anxiety: [0.62734448 0.44799421 0.66263027 0.87712665 0.7472837  0.86795348
 0.47992046]
Mild: [0.79790563 0.59843579 0.25462673 0.92181273 0.82909197 0.90005441
 0.53036814]
Moderate: [0.74251839 0.61407053 0.79659025 0.90707691 0.76764947 0.79057245
 0.41603317]
Severe: [0.91388944 0.73406422 0.12004501 0.8867539  0.84960712 0.93434534
 0.45857136]


In [132]:
# get the mean for each vector
print("Anxiety:", anxiety_vector.mean())
print("Mild:", mild_vector.mean())
print("Moderate:", moderate_vector.mean())
print("Severe:", severe_vector.mean())

Anxiety: 0.6728933226082707
Mild: 0.6903279141328051
Moderate: 0.7192158809398054
Severe: 0.6996109140961436


### Getting the Consine Similarity for the test input video


In [134]:
df_s01 = pd.read_csv("S01_1001-1.csv")
df_s01.columns = [c.strip() for c in df_s01.columns]
df_s01.head()

,frame,face_id,timestamp,confidence,success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.98,1,1.40,1.52,1.40,0.0,0.52,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,0,0.008,0.98,1,1.20,0.72,1.16,0.0,0.68,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,3,0,0.017,0.98,1,1.25,0.70,1.08,0.0,0.72,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,4,0,0.025,0.88,1,1.39,0.95,1.07,0.0,0.81,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,5,0,0.033,0.98,1,1.72,1.16,1.16,0.0,0.83,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [135]:
# get the duration of the video
duration = df_s01["timestamp"].iloc[-1]
duration

17.901

In [136]:
# get the number of FAUs that appeared based on the number of frames
for au in FAU:
    cond1 = df_s01[au + "_c"] == 1
    cond2 = df_s01[au + "_r"] >= 0.5
    cond3 = df_s01["confidence"] >= 0.98
    cond4 = df_s01["success"] == 1
    df_s01[au] = np.where(cond1 & cond2 & cond3 & cond4, 1, 0)

df_s01

,frame,face_id,timestamp,confidence,success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,...,AU45_c,AU01,AU02,AU04,AU06,AU12,AU15,AU17,AU20,AU23
0,1,0,0.000,0.98,1,1.40,1.52,1.40,0.0,0.52,...,1.0,1,1,1,0,1,0,0,0,1
1,2,0,0.008,0.98,1,1.20,0.72,1.16,0.0,0.68,...,1.0,1,1,1,0,0,0,0,0,1
2,3,0,0.017,0.98,1,1.25,0.70,1.08,0.0,0.72,...,1.0,1,1,1,0,0,0,0,0,0
3,4,0,0.025,0.88,1,1.39,0.95,1.07,0.0,0.81,...,1.0,0,0,0,0,0,0,0,0,0
4,5,0,0.033,0.98,1,1.72,1.16,1.16,0.0,0.83,...,1.0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2142,2143,0,17.868,0.98,1,0.00,0.00,1.05,0.0,0.74,...,0.0,0,0,0,0,0,0,0,0,0
2143,2144,0,17.876,0.98,1,0.06,0.01,1.04,0.0,0.82,...,0.0,0,0,0,0,0,0,0,0,0
2144,2145,0,17.885,0.98,1,0.06,0.01,0.99,0.0,0.77,...,0.0,0,0,0,0,0,0,0,0,0
2145,2146,0,17.893,0.98,1,0.11,0.01,1.17,0.0,0.78,...,0.0,0,0,0,0,0,0,0,0,0


In [137]:
df_s01_filtered = np.ceil(df_s01[FAU].sum() / duration)
df_s01_filtered

AU01    33.0
AU02     2.0
AU04    81.0
AU06     0.0
AU12     1.0
AU15     0.0
AU17     1.0
AU20     2.0
AU23     4.0
dtype: float64

In [138]:
df_s01_array = np.array(df_s01_filtered).reshape(1, -1)
df_s01_array

array([[33.,  2., 81.,  0.,  1.,  0.,  1.,  2.,  4.]])

In [139]:
# compare the input vector with all 28 vectors
anxiety_vector = []
mild_vector = []
moderate_vector = []
severe_vector = []
for i in range(len(df_anxiety)):
    anxiety = cosine_similarity(df_anxiety[i].reshape(1, -1), df_s01_array)
    mild = cosine_similarity(df_mild[i].reshape(1, -1), df_s01_array)
    moderate = cosine_similarity(df_moderate[i].reshape(1, -1), df_s01_array)
    severe = cosine_similarity(df_severe[i].reshape(1, -1), df_s01_array)
    anxiety_vector = np.append(anxiety_vector, anxiety)
    mild_vector = np.append(mild_vector, mild)
    moderate_vector = np.append(moderate_vector, moderate)
    severe_vector = np.append(severe_vector, severe)

print("Anxiety:", anxiety_vector)
print("Mild:", mild_vector)
print("Moderate:", moderate_vector)
print("Severe:", severe_vector)

Anxiety: [0.54895912 0.35626897 0.60666392 0.81033996 0.60102408 0.76096808
 0.28484239]
Mild: [0.74077253 0.5362405  0.19783014 0.85158786 0.69968632 0.75588734
 0.33682617]
Moderate: [0.65147559 0.57378793 0.82826848 0.80147724 0.59859594 0.58728889
 0.2028651 ]
Severe: [0.90980024 0.66815851 0.04597581 0.7401755  0.73886061 0.91069127
 0.28267893]


In [140]:
# get the mean for each vector
print("Anxiety:", anxiety_vector.mean())
print("Mild:", mild_vector.mean())
print("Moderate:", moderate_vector.mean())
print("Severe:", severe_vector.mean())

Anxiety: 0.5670095023217293
Mild: 0.5884044086876293
Moderate: 0.6062513095806461
Severe: 0.613762982397451
